In [1]:
import mysql.connector

def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Root@123#",
        database="HospitalManagement"
    )

conn = get_connection()
cursor = conn.cursor()

# Add column if not exists
try:
    cursor.execute("""
        ALTER TABLE Appointment
        ADD COLUMN IsReadmission BOOLEAN DEFAULT 0
    """)
    print("IsReadmission column added.")
except:
    print("IsReadmission column already exists.")

# Update readmission status (same random data, same 1000 rows)
cursor.execute("""
    UPDATE Appointment a
    JOIN (
        SELECT 
            AppointmentID,
            ROW_NUMBER() OVER (
                PARTITION BY PatientID 
                ORDER BY AppointmentDate
            ) AS visit_no
        FROM Appointment
    ) ranked
    ON a.AppointmentID = ranked.AppointmentID
    SET a.IsReadmission =
        CASE
            WHEN ranked.visit_no > 1 THEN 1
            ELSE 0
        END;
""")

conn.commit()
conn.close()

print("Readmission status updated in Appointment table (1000 rows).")


IsReadmission column already exists.
Readmission status updated in Appointment table (1000 rows).


In [3]:
import pandas as pd
import mysql.connector
import os
import time

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Root@123#",
    database="HospitalManagement"
)

# =====================================================
# APPOINTMENT DATA (1000 ROWS) WITH TOTAL READMISSIONS
# =====================================================
query = """
SELECT 
    a.AppointmentID,
    a.PatientID,
    p.Name AS PatientName,
    a.DoctorID,
    dpt.DepartmentName,
    a.AppointmentDate,
    a.Status,

    CASE 
        WHEN a.IsReadmission = 1 THEN 'Yes'
        ELSE 'No'
    END AS ReadmissionStatus,

    -- Total readmissions per patient (derived, not separate table)
    SUM(a.IsReadmission) OVER (PARTITION BY a.PatientID) 
        AS TotalReadmissions

FROM Appointment a
JOIN Patient p ON a.PatientID = p.PatientID
JOIN Doctor d ON a.DoctorID = d.DoctorID
JOIN Department dpt ON d.DepartmentID = dpt.DepartmentID
ORDER BY a.AppointmentID;
"""

df = pd.read_sql(query, conn)

pd.set_option("display.max_rows", None)
print("\n📋 Appointment Table (1000 rows) with Readmission Info")
display(df)

# =====================================================
# DEPARTMENT RESPONSIBLE FOR MOST READMISSIONS
# =====================================================
dept_query = """
SELECT 
    dpt.DepartmentName,
    COUNT(*) AS TotalReadmissions
FROM Appointment a
JOIN Doctor d ON a.DoctorID = d.DoctorID
JOIN Department dpt ON d.DepartmentID = dpt.DepartmentID
WHERE a.IsReadmission = 1
GROUP BY dpt.DepartmentName
ORDER BY TotalReadmissions DESC;
"""

dept_df = pd.read_sql(dept_query, conn)

print("\n🏥 Departments Handling the Most Readmissions")
display(dept_df)

# =====================================================
# EXPORT TO EXCEL
# =====================================================
folder = "DBMS_Output"
os.makedirs(folder, exist_ok=True)

excel_path = f"{folder}/Appointment_Readmission_Analysis_{time.strftime('%Y%m%d_%H%M%S')}.xlsx"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Appointments_1000_Rows", index=False)
    dept_df.to_excel(writer, sheet_name="Department_Readmissions", index=False)

print(f"\n✅ Excel sheet created at:\n{excel_path}")

conn.close()


C:\Users\sonka\AppData\Local\Temp\ipykernel_21596\4166429214.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



📋 Appointment Table (1000 rows) with Readmission Info


,AppointmentID,PatientID,PatientName,DoctorID,DepartmentName,AppointmentDate,Status,ReadmissionStatus,TotalReadmissions
0,1,199,Vikash,19,Orthopedics,2025-12-06,Scheduled,Yes,3.0
1,2,800,Karan,49,Orthopedics,2025-11-07,Scheduled,Yes,3.0
2,3,929,Nikhil,38,Orthopedics,2025-06-02,Scheduled,No,0.0
3,4,752,Sidharth,8,General Medicine,2025-07-19,Completed,No,0.0
4,5,164,Sneha,42,Neurology,2025-12-08,Completed,No,0.0
5,6,602,Neha,22,Cardiology,2025-02-02,Scheduled,No,0.0
6,7,79,Nikhil,19,Orthopedics,2025-08-03,Scheduled,No,0.0
7,8,173,Nikhil,10,Cardiology,2025-01-20,Scheduled,No,3.0
8,9,800,Karan,16,General Medicine,2025-11-12,Completed,Yes,3.0
9,10,733,Vikash,50,Pediatrics,2025-12-22,Completed,Yes,2.0



🏥 Departments Handling the Most Readmissions


C:\Users\sonka\AppData\Local\Temp\ipykernel_21596\4166429214.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dept_df = pd.read_sql(dept_query, conn)


,DepartmentName,TotalReadmissions
0,General Medicine,96
1,Cardiology,72
2,Pediatrics,71
3,Orthopedics,70
4,Neurology,63



✅ Excel sheet created at:
DBMS_Output/Appointment_Readmission_Analysis_20260122_025623.xlsx
